**NOT&#258;**
C&#226;nd utiliz&#259;m [Binder](https://mybinder.org/), comanda ``python -m pip install --user numpy plotly`` trebuie rulat&#259; &#238;n terminal &#238;nainte de pornirea **notebook**-ului!

# Separatoare &#238;n planul fazelor

**Separatoarele** sunt curbe care separ&#259; zonele *cu dinamic&#259; diferit&#259;*. Ele sunt grani&#355;a dintre orbitele &#238;nchise &#351;i cele nem&#259;rginite. 

In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

In [ ]:
from decimal import Context, ROUND_HALF_EVEN, FloatOperation, setcontext, Decimal
contextul = Context(prec=128, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=1,
                    flags=[], traps=[FloatOperation])
setcontext(contextul)

In [ ]:
init_notebook_mode(connected=True)

Construim aproxim&#259;ri ale constantei $\pi$ &#351;i ale func&#355;iilor **sin**, **exp**, conform [recomand&#259;rii oficiale](https://docs.python.org/3/library/decimal.html#recipes):

In [ ]:
def pi():
    '''
    Compute Pi to the current precision.

    >>> print(pi())
    3.141592653589793238462643383
    '''
    contextul.prec += 2     # extra digits for intermediate steps
    three = Decimal(3)      # substitute "three=3.0" for regular floats
    lasts, t, s, n, na, d, da = 0, three, 3, 1, 0, 0, 24
    while s != lasts:
        lasts = s
        n, na = n+na, na+8
        d, da = d+da, da+32
        t = (t * n) / d
        s += t
    contextul.prec -= 2
    return +s               # unary plus applies the new precision

def exp(x):
    '''
    Return e raised to the power of x.  Result type matches input type.

    >>> print(exp(Decimal(1)))
    2.718281828459045235360287471
    >>> print(exp(Decimal(2)))
    7.389056098930650227230427461
    >>> print(exp(2.0))
    7.38905609893
    >>> print(exp(2+0j))
    (7.38905609893+0j)
    '''
    contextul.prec += 2
    i, lasts, s, fact, num = 0, 0, 1, 1, 1
    while s != lasts:
        lasts = s
        i += 1
        fact *= i
        num *= x
        s += num / fact
    contextul.prec -= 2
    return +s

def sin(x):
    '''
    Return the sine of x as measured in radians.

    The Taylor series approximation works best for a small value of x.
    For larger values, first compute x = x % (2 * pi).

    >>> print(sin(Decimal('0.5')))
    0.4794255386042030002732879352
    >>> print(sin(0.5))
    0.479425538604
    >>> print(sin(0.5+0j))
    (0.479425538604+0j)
    '''
    contextul.prec += 2
    i, lasts, s, fact, num, sign = 1, 0, x, 1, x, 1
    
    x = x % (2 * pi())      # pentru numere mari
    
    while s != lasts:
        lasts = s
        i += 2
        fact *= i * (i-1)
        num *= x * x
        sign *= -1
        s += num / fact * sign
    contextul.prec -= 2
    return +s

Calcule privind matricea $A\in\mathcal{M}_{2}(\mathbb{R})$
----------------------------------------------------------

In [ ]:
a11 = Decimal('1.0')
a12 = Decimal('0.0')
a21 = Decimal('0.0')
a22 = Decimal('-2.0')
matricea_A = np.array([[a11, a12], [a21, a22]], dtype=np.object_)
matricea_Af = np.array([[float(a11), float(a12)], [float(a21), float(a22)]])

Pentru calcule cu **float** ([floating-point numbers](https://en.wikipedia.org/wiki/Floating-point_arithmetic)) de &#238;nalt&#259; precizie, vezi [mpmath](http://mpmath.org/).

Elemente spectrale ale matricei $A$
-----------------------------------

Urma ([trace](https://en.wikipedia.org/wiki/Trace_(linear_algebra))):

In [ ]:
Trace_A = np.trace(matricea_A)
Trace_Af = np.trace(matricea_Af)
print("Trace(A) = {0:.16f}, Trace(Af) = {1:.16f}".format(Trace_A, Trace_Af))

Valorile proprii ([eigenvalues](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors)), vectorii proprii ([eigenvectors](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors#Eigenspaces,_geometric_multiplicity,_and_the_eigenbasis_for_matrices)):

In [ ]:
([lambda1, lambda2], [v1, v2]) = np.linalg.eig(matricea_Af)
print("Valorile proprii ale matricei Af:\n"
      "lambda1 = {0:.16f}\nlambda2 = {1:.16f}"
      "\nVectorii proprii:\n"
      "v1 = ({2:.16f}, {3:.16f})\n"
      "v2 = ({4:.16f}, {5:.16f})".format(lambda1, lambda2, v1[0], v1[1], v2[0], v2[1]))

Folosind eigenvectorii, construim **datele Cauchy** ale solu&#355;iilor $\mbox{e}^{\lambda_{i}t}\cdot\overline{v}_{i}$, unde $i\in\overline{1,2}$.

Determinantul, vezi [det](https://github.com/numpy/numpy/blob/master/numpy/linalg/linalg.py#L2103):

In [ ]:
Det_Af = np.linalg.det(matricea_Af)
print("det Af = {0:.16f}".format(Det_Af))

Formulele ecua&#355;iilor din sistemul diferen&#355;ial:

In [ ]:
def sist_dif(X, Y, Z, t):
    X_loc = Decimal(X)
    Y_loc = Decimal(Y)
    Z_loc = Decimal(Z)
    T_loc = Decimal(t)
    
    # pendulul matematic: (D. Jordan, P. Smith, NODE, IV-th Ed., pag. 12)
    '''
    ec_X = Y_loc
    ec_Y = -sin(X_loc)
    ec_Z = Decimal('0')
    ec_T = T_loc  
    '''
    
    # resort neliniar: (D. Jordan, P. Smith, NODE, IV-th Ed., pag. 19)
    '''
    ec_X = Y_loc
    ec_Y = -X_loc * exp(-X_loc)  # sau -X_loc * ((-X_loc).exp()), vezi https://docs.python.org/3/library/decimal.html#decimal.Decimal.exp
    ec_Z = Decimal('0')
    ec_T = T_loc
    '''
    
    # sistem liniar: (D. Jordan, P. Smith, NODE, IV-th Ed., pag. 63 si urm.)
    
    a11 = Decimal('1.0')
    a12 = Decimal('0.0')
    a21 = Decimal('0.0')
    a22 = Decimal('-2.0')
    ec_X = a11 * X_loc + a12 * Y_loc
    ec_Y = a21 * X_loc + a22 * Y_loc
    ec_Z = Decimal('0')
    ec_T = T_loc
    
    
    return ec_X, ec_Y, ec_Z, ec_T

Pasul $h$ este:

In [ ]:
h = Decimal('0.01')
h2 = Decimal('0.5') * h
h3 = h / Decimal('6.0')

Num&#259;rul de itera&#355;ii $n$ este:

In [ ]:
n = 10_000

# 6_000 pentru pendulul matematic
# 1_000 pentru resortul neliniar
# 10_000 pentru sistemul liniar

In [ ]:
r2 = range(n)
r3 = range(n + 1)

X = np.array(r3, dtype=np.object_)
Y = np.array(r3, dtype=np.object_)
Z = np.array(r3, dtype=np.object_)
t = np.array(r3, dtype=np.object_)

kX1 = np.array(r2, dtype=np.object_)
kY1 = np.array(r2, dtype=np.object_)
kZ1 = np.array(r2, dtype=np.object_)
zX1 = np.array(r2, dtype=np.object_)
zY1 = np.array(r2, dtype=np.object_)
zZ1 = np.array(r2, dtype=np.object_)

kX2 = np.array(r2, dtype=np.object_)
kY2 = np.array(r2, dtype=np.object_)
kZ2 = np.array(r2, dtype=np.object_)
zX2 = np.array(r2, dtype=np.object_)
zY2 = np.array(r2, dtype=np.object_)
zZ2 = np.array(r2, dtype=np.object_)

kX3 = np.array(r2, dtype=np.object_)
kY3 = np.array(r2, dtype=np.object_)
kZ3 = np.array(r2, dtype=np.object_)
zX3 = np.array(r2, dtype=np.object_)
zY3 = np.array(r2, dtype=np.object_)
zZ3 = np.array(r2, dtype=np.object_)

kX4 = np.array(r2, dtype=np.object_)
kY4 = np.array(r2, dtype=np.object_)
kZ4 = np.array(r2, dtype=np.object_)

kXRK4 = np.array(r2, dtype=np.object_)
kYRK4 = np.array(r2, dtype=np.object_)
kZRK4 = np.array(r2, dtype=np.object_)

In [ ]:
X[0], Y[0], Z[0], t[0] = (Decimal('0.01'), Decimal('0.0'), Decimal('1.0'), Decimal('0.0'))

# pendulul matematic: ochi de pisica (Decimal('0.0'), Decimal('2.0'), Decimal('1.0'), Decimal('0.0'))
# pendulul matematic: orbita deschisa (Decimal('0.0'), Decimal('2.16'), Decimal('1.0'), Decimal('0.0'))
# pendulul matematic: orbita inchisa (Decimal('0.0'), Decimal('1.9'), Decimal('1.0'), Decimal('0.0'))

# resortul neliniar: orbita deschisa (Decimal('8.751035'), Decimal('-1.7280'), Decimal('1.0'), Decimal('0.0'))
# resortul neliniar: orbita inchisa (Decimal('0.0'), Decimal('1.0'), Decimal('1.0'), Decimal('0.0'))

# sistemul liniar (1,0,0,-2): orbita rectilinie (pentru eigenvaloarea pozitiva) (Decimal('0.0'), Decimal('1.0'), Decimal('1.0'), Decimal('0.0'))
# sistemul liniar (1,0,0,-2): orbita rectilinie (pentru eigenvaloarea negativa) (Decimal('0.01'), Decimal('0.0'), Decimal('1.0'), Decimal('0.0'))

In [ ]:
for i in r2:
    i2 = i + 1 
    ti = t[i]
    ti2 = ti + h2
    ti3 = ti + h
    
    t[i2] = ti3                                                           # t[i+1] = t[i] + h
    
    
    kX1[i], kY1[i], kZ1[i], ti = sist_dif(X[i], Y[i], Z[i], ti)           # [pasul 1]
    zX1[i] = X[i] + h2 * kX1[i]
    zY1[i] = Y[i] + h2 * kY1[i]
    zZ1[i] = Z[i] + h2 * kZ1[i]
    
    kX2[i], kY2[i], kZ2[i], ti2 = sist_dif(zX1[i], zY1[i], zZ1[i], ti2)   # [pasul 2]
    zX2[i] = X[i] + h2 * kX2[i]
    zY2[i] = Y[i] + h2 * kY2[i]
    zZ2[i] = Z[i] + h2 * kZ2[i]
    
    kX3[i], kY3[i], kZ3[i], ti2 = sist_dif(zX2[i], zY2[i], zZ2[i], ti2)   # [pasul 3]
    zX3[i] = X[i] + h * kX3[i]
    zY3[i] = Y[i] + h * kY3[i]
    zZ3[i] = Z[i] + h * kZ3[i]
    
    kX4[i], kY4[i], kZ4[i], ti3 = sist_dif(zX3[i], zY3[i], zZ3[i], ti3)   # [pasul 4]
    
    kXRK4[i] = h3 * (kX1[i] + 2 * kX2[i] + 2 * kX3[i] + kX4[i])           # [pasul 5]
    kYRK4[i] = h3 * (kY1[i] + 2 * kY2[i] + 2 * kY3[i] + kY4[i])
    kZRK4[i] = h3 * (kZ1[i] + 2 * kZ2[i] + 2 * kZ3[i] + kZ4[i])
    
    X[i2] = X[i] + kXRK4[i]
    Y[i2] = Y[i] + kYRK4[i]
    Z[i2] = Z[i] + kZRK4[i]    
    
    print("+++\nN = {0:d}, "
          "x({1}) = {2:.16f}, y({3}) = {4:.16f}, z({5}) = {6:.16f}".format(i, i, X[i], i, Y[i], i, Z[i]))

La itera&#355;ia num&#259;rul $J$, unde

In [ ]:
J = 564
# 803 pentru pendulul matematic
# 564 pentru resortul neliniar

au fost ob&#355;inute valorile:

In [ ]:
print("x({0:d}) = {1:.16f}, y({2:d}) = {3:.16f}, z({4:d}) = {5:.16f}".format(J, X[J], J, Y[J], J, Z[J]))

Preg&#259;tirea vizualiz&#259;rii:

In [ ]:
traiectoria = go.Scatter3d(
# imaginea va putea fi salvata doar ca "png", vezi nota din "https://plot.ly/python/static-image-export/#vector-formats-svg-and-pdf"
              x=X, y=Y, z=Z, mode='markers', 
              marker=dict(color='rgb(127,0,255)', size=4, symbol='circle',
                          line=dict(color='rgb(255,245,0)', width=1),
                          opacity=0.8))
datele = [traiectoria]
aspectul = go.Layout(margin=dict(l=0, r=0, b=0, t=0))

Vizualizarea interativ&#259; a datelor:

In [ ]:
fig = go.Figure(data=datele, layout=aspectul)
iplot(fig)